In [1]:
import psycopg2, os, getpass
import pandas as pd
import numpy as py


In [64]:
df = pd.read_csv('Downloads/airlines.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4408 entries, 0 to 4407
Data columns (total 24 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   Airport.Code                                         4408 non-null   object
 1   Airport.Name                                         4408 non-null   object
 2   Time.Label                                           4408 non-null   object
 3   Time.Month                                           4408 non-null   int64 
 4   Time.Month Name                                      4408 non-null   object
 5   Time.Year                                            4408 non-null   int64 
 6   Statistics.# of Delays.Carrier                       4408 non-null   int64 
 7   Statistics.# of Delays.Late Aircraft                 4408 non-null   int64 
 8   Statistics.# of Delays.National Aviation System      4408 non-null   int64 
 9

In [8]:
def clean_column_names(list_of_cols):
    return [(item.lower()
             .replace("statistics.","")
             .replace("#","num")
             .replace(" ", "_")
             .replace(".","_")
            )
             for item in list_of_cols]
    

In [108]:
dir(df)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_e

In [105]:
df.columns = clean_column_names(df.columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4408 entries, 0 to 4407
Data columns (total 24 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   airport_code                              4408 non-null   object
 1   airport_name                              4408 non-null   object
 2   time_label                                4408 non-null   object
 3   time_month                                4408 non-null   int64 
 4   time_month_name                           4408 non-null   object
 5   time_year                                 4408 non-null   int64 
 6   num_of_delays_carrier                     4408 non-null   int64 
 7   num_of_delays_late_aircraft               4408 non-null   int64 
 8   num_of_delays_national_aviation_system    4408 non-null   int64 
 9   num_of_delays_security                    4408 non-null   int64 
 10  num_of_delays_weather                     4408 n

In [ ]:
#postgres can't adapt type 'numpy.int64'

In [76]:
postgrestypes = ['TEXT', 'TEXT', 'TEXT', 'INTEGER', 'INTEGER',
 'INTEGER', 'INTEGER','INTEGER','INTEGER','INTEGER',
 'INTEGER','TEXT', 'INTEGER','INTEGER','INTEGER','INTEGER','INTEGER','INTEGER',
 'INTEGER','INTEGER','INTEGER','INTEGER','INTEGER','INTEGER']

In [77]:
def table_creation_string(col_names, postgres_types):

    def parse(col_names, postgres_types):
        table = []
        if len(col_names) != len(postgres_types): assert False
        for index,_ in enumerate(col_names):
            table.append("{} {}".format(col_names[index],postgres_types[index]))
        return table
    
    return "CREATE TABLE airlines({});".format(str.join(", ",
                                              parse(col_names, postgres_types)))



In [78]:
table_creation_string(df.columns, postgrestypes)

'CREATE TABLE airlines(airport_code TEXT, airport_name TEXT, time_label TEXT, time_month INTEGER, time_month_name INTEGER, time_year INTEGER, num_of_delays_carrier INTEGER, num_of_delays_late_aircraft INTEGER, num_of_delays_national_aviation_system INTEGER, num_of_delays_security INTEGER, num_of_delays_weather INTEGER, carriers_names TEXT, carriers_total INTEGER, flights_cancelled INTEGER, flights_delayed INTEGER, flights_diverted INTEGER, flights_on_time INTEGER, flights_total INTEGER, minutes_delayed_carrier INTEGER, minutes_delayed_late_aircraft INTEGER, minutes_delayed_national_aviation_system INTEGER, minutes_delayed_security INTEGER, minutes_delayed_total INTEGER, minutes_delayed_weather INTEGER);'

In [ ]:
conn = psycopg2.connect(database="jupyterdb", user=getpass.getuser())

In [106]:
cur = conn.cursor()
conn.autocommit = True
cur.execute("DROP TABLE IF EXISTS airlines;")
cur.execute(table_creation_string(df.columns, postgrestypes))
cur.close()

In [107]:
two_rows = df.iloc[:2]
two_rows

,airport_code,airport_name,time_label,time_month,time_month_name,time_year,num_of_delays_carrier,num_of_delays_late_aircraft,num_of_delays_national_aviation_system,num_of_delays_security,...,flights_delayed,flights_diverted,flights_on_time,flights_total,minutes_delayed_carrier,minutes_delayed_late_aircraft,minutes_delayed_national_aviation_system,minutes_delayed_security,minutes_delayed_total,minutes_delayed_weather
0,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2003/06,6,June,2003,1009,1275,3217,17,...,5843,27,23974,30060,61606,68335,118831,518,268764,19474
1,BOS,"Boston, MA: Logan International",2003/06,6,June,2003,374,495,685,3,...,1623,3,7875,9639,20319,28189,24400,99,77167,4160


In [89]:
def table_insertion_string(df):
    pass

In [90]:
string = "INSERT INTO airlines VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s),(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) "

In [96]:
data = df.iloc[0].to_list(),df.iloc[1].to_list()

In [97]:
cur = conn.cursor()
cur.execute(string, data)
cur.close()

ProgrammingError: can't adapt type 'numpy.int64'